In [27]:
!pip install pandas numpy matplotlib tqdm fastdtw seaborn networkx

In [ ]:
!pip install scikit-learn


In [ ]:
import torch

!pip uninstall torch-scatter torch-sparse torch-geometric torch-cluster  --y
!pip install torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git

In [ ]:
!pip install torch-geometric torch-geometric-temporal

In [ ]:
import sys
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
from scipy.stats import pearsonr, zscore, rankdata
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from torch_geometric.data import Data, Batch
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GATConv, global_mean_pool
from torch_geometric.nn import global_mean_pool
from torch_geometric_temporal.nn.recurrent import GConvGRU, DCRNN
from torch_geometric.utils import from_networkx, add_self_loops, to_undirected
from tqdm import tqdm
from torch.utils.data import Dataset
from torch_geometric.data import Dataset



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
lines = []
with open('/content/drive/MyDrive/EP1.01.txt', 'r') as file:
    for line in file:
        lines.append(line)
print(len(lines))

In [ ]:
print(lines[0])
print(lines[1])
print(lines[2])
print(lines[3])
print(lines[4])

In [ ]:
split_ = lines[0].split()
for i in range(len(split_)):
    print(i, split_[i])

In [ ]:
event = []
digit = []
pos = []
data = []
line_counter = 0

data_df = pd.DataFrame(columns=['event', 'digit', 'pos', 'data'])

for line in tqdm(lines):
    if line_counter<14_000_000:
        split_line = line.split()
        event.append(split_line[1])
        digit.append(split_line[4])
        pos.append(split_line[3])
        data.append(split_line[6])
        line_counter+=1

data_df['event'] = event
data_df['digit'] = digit
data_df['pos'] = pos
data_df['data'] = data

num_events = data_df['event'].unique()
print(len(num_events))

del lines, event, digit, pos, data, num_events

print(data_df.head())
print(data_df.shape)

In [ ]:
data_df['data'] = data_df['data'].apply(lambda x: [float(i) for i in x.split(',')])
data_df['data'] = data_df['data'].apply(lambda x: x[:250])

data_df['event'] = data_df['event'].apply(float)
data_df['digit'] = data_df['digit'].apply(float)
data_df['digit'] = data_df['digit'].apply(lambda x: 10 if x == -1 else x)

scaler = StandardScaler()
data_df['data'] = data_df['data'].apply(lambda x: scaler.fit_transform(np.array(x).reshape(-1, 1)).flatten())

# data_df.to_csv('data_df.csv')

print(data_df.head(28))
print(data_df.shape)


In [ ]:
pos_values = data_df['pos'].unique()
print(pos_values)
data_lengths = data_df['data'].apply(len)
unique_lengths = data_lengths.unique()
print(unique_lengths)
del data_lengths

In [ ]:
def calculate_correlation(series1, series2):
    return pearsonr(series1, series2)[0]

def calculate_euclidean(series1, series2):
    return euclidean(series1, series2)

def calculate_dtw(series1, series2):
    distance, path = fastdtw(series1, series2)
    return distance

probes = data_df['pos'].unique()
similarity_matrix_corr = pd.DataFrame(index=probes, columns=probes)
similarity_matrix_euc = pd.DataFrame(index=probes, columns=probes)
similarity_matrix_dtw = pd.DataFrame(index=probes, columns=probes)


for i, probe1 in enumerate(probes):
    for j, probe2 in enumerate(probes):
        if i < j:
            series1 = data_df[data_df['pos'] == probe1]['data'].iloc[0]
            series2 = data_df[data_df['pos'] == probe2]['data'].iloc[0]

            corr = calculate_correlation(series1, series2)
            euc = calculate_euclidean(series1, series2)
            dtw = calculate_dtw(series1, series2)

            similarity_matrix_corr.at[probe1, probe2] = corr
            similarity_matrix_euc.at[probe2, probe1] = euc
            similarity_matrix_dtw.at[probe1, probe2] = dtw

np.fill_diagonal(similarity_matrix_corr.values, 0)
np.fill_diagonal(similarity_matrix_euc.values, 0)
np.fill_diagonal(similarity_matrix_dtw.values, 0)

print(similarity_matrix_corr)
print(similarity_matrix_euc)
print(similarity_matrix_dtw)


In [ ]:
def compare_sim_mat(similarity_matrix):
    similarity_matrix = similarity_matrix.apply(pd.to_numeric, errors='coerce')

    similarity_matrix = similarity_matrix.where(pd.notna(similarity_matrix), similarity_matrix.T)

    range_val = similarity_matrix.max().max() - similarity_matrix.min().min()
    if range_val == 0:
        normalized_matrix = similarity_matrix
    else:
        normalized_matrix = (similarity_matrix - similarity_matrix.min().min()) / range_val

    standardized_matrix = similarity_matrix.apply(lambda x: (x - x.mean()) / x.std(), axis=1)

    ranked_matrix = similarity_matrix.rank(method='average')
    ranked_matrix = (ranked_matrix - ranked_matrix.min().min()) / (ranked_matrix.max().max() - ranked_matrix.min().min())

    fig, axes = plt.subplots(1, 3, figsize=(18, 6))

    sns.heatmap(normalized_matrix, annot=True, ax=axes[0])
    axes[0].set_title('Normalized Similarity Matrix')

    sns.heatmap(standardized_matrix, annot=True, ax=axes[1])
    axes[1].set_title('Standardized Similarity Matrix')

    sns.heatmap(ranked_matrix, annot=True, ax=axes[2])
    axes[2].set_title('Ranked Similarity Matrix')

    plt.tight_layout()
    plt.show()

compare_sim_mat(similarity_matrix_corr)
compare_sim_mat(similarity_matrix_euc)
compare_sim_mat(similarity_matrix_dtw)



In [ ]:
coordinates = {
    'AF3': {'phi': (129.9+89.7)/2,
            'theta': (52.2+41.0)/2},
    'F7': {'phi': 137.2,
            'theta': 83.1},
    'F3': {'phi': 129.9,
            'theta': 52.2},
    'FC5': {'phi': (137.2+129.9+173.9+180)/4 ,
            'theta': (83.1+52.2+95+45.2/4)},
    'T7': {'phi': 173.9,
            'theta': 95.0},
    'P7': {'phi': 216.1,
            'theta': 92.9},
    'O1': {'phi': 250.6,
            'theta': 89.2},
    'O2': {'phi': 287.5,
            'theta': 90.1},
    'P8': {'phi': 322.7,
            'theta': 94.9},
    'T8': {'phi': 3.2,
            'theta': 95.8},
    'FC6': {'phi': (-1.0+3.2+40.3+49.8)/4,
            'theta':(46.0+95.8+84.1+53.6)/4 },
    'F4': {'phi': 49.8,
            'theta': 53.6},
    'F8': {'phi': 40.3,
            'theta': 84.1},
    'AF4': {'phi': (89.7+49.8)/2,
            'theta': (41.0+53.6)/2}
}

In [ ]:
def spherical_to_cartesian(phi, theta):
    phi_rad = np.radians(phi)
    theta_rad = np.radians(theta)
    x = np.cos(phi_rad) * np.sin(theta_rad)
    y = np.sin(phi_rad) * np.sin(theta_rad)
    z = np.cos(theta_rad)
    return x, y, z

adjacency_matrix = np.zeros((len(coordinates), len(coordinates)))

for i, (coord_i, values_i) in enumerate(coordinates.items()):
    x_i, y_i, z_i = spherical_to_cartesian(values_i['phi'], values_i['theta'])
    for j, (coord_j, values_j) in enumerate(coordinates.items()):
        if i != j:
            x_j, y_j, z_j = spherical_to_cartesian(values_j['phi'], values_j['theta'])
            d = np.sqrt((x_j - x_i) ** 2 + (y_j - y_i) ** 2 + (z_j - z_i) ** 2)
            w = 1 / np.sqrt(d)
            adjacency_matrix[i, j] = w

max_weight = adjacency_matrix.max()
adjacency_matrix[adjacency_matrix > 0] /= max_weight

np.fill_diagonal(adjacency_matrix, 1)

fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(adjacency_matrix, annot=True, cmap='viridis', ax=ax)

plt.title('EEG Electrodes Weighted Adjacency Matrix')
plt.xlabel('Electrodes')
plt.ylabel('Electrodes')
plt.xticks(ticks=np.arange(len(coordinates)) + 0.5, labels=list(coordinates.keys()), rotation=45)
plt.yticks(ticks=np.arange(len(coordinates)) + 0.5, labels=list(coordinates.keys()), rotation=45)

plt.show()


In [ ]:
def calculate_weights(coordinates):

    num_nodes = len(coordinates)
    adjacency_matrix = np.zeros((num_nodes, num_nodes))

    for i, coord_i in enumerate(coordinates.keys()):
        values_i = coordinates[coord_i]
        x_i, y_i, z_i = spherical_to_cartesian(values_i['phi'], values_i['theta'])
        for j, coord_j in enumerate(coordinates.keys()):
            if i != j:
                values_j = coordinates[coord_j]
                x_j, y_j, z_j = spherical_to_cartesian(values_j['phi'], values_j['theta'])
                d = np.sqrt((x_j - x_i) ** 2 + (y_j - y_i) ** 2 + (z_j - z_i) ** 2)
                w = 1 / d
                adjacency_matrix[i, j] = w

    max_weight = adjacency_matrix.max()
    adjacency_matrix[adjacency_matrix > 0] /= max_weight
    np.fill_diagonal(adjacency_matrix, 1)

    return adjacency_matrix



In [ ]:
class EEGGraphDataset(torch.utils.data.Dataset):
    def __init__(self, dataframe, edge_index, edge_weight):
        self.dataframe = dataframe
        self.edge_index = edge_index
        self.edge_weight = edge_weight
        self.edge_index = to_undirected(self.edge_index)
        self.edge_index, _ = add_self_loops(self.edge_index)
        self.unique_events = dataframe['event'].unique()

    def __len__(self):
        return len(self.unique_events)

    def __getitem__(self, idx):
        event_id = self.unique_events[idx]
        event_data = self.dataframe[self.dataframe['event'] == event_id]
        sequence = []
        for time_step in range(len(event_data.iloc[0]['data'])):
            x = torch.tensor([row.data[time_step] for row in event_data.itertuples(index=False)], dtype=torch.float).unsqueeze(1)
            snapshot = Data(x=x, edge_index=self.edge_index, edge_attr=self.edge_weight)
            sequence.append(snapshot)
        label = event_data.iloc[0]['digit']
        return sequence, label


In [ ]:
def collate_graph_sequences(batch):
    sequences, labels = zip(*batch)
    batched_sequences = [Batch.from_data_list(sequence) for sequence in sequences]
    return batched_sequences, torch.tensor(labels)


In [ ]:
for name, size in sorted(((name, sys.getsizeof(value)) for name, value in globals().items()),
                         key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name, size))


In [ ]:
from torch.utils.data import Subset

def custom_train_test_split(dataset, test_size=0.2, random_state=42):
    np.random.seed(random_state)

    indices = np.arange(len(dataset))
    print(indices.size)
    np.random.shuffle(indices)

    split_idx = int(len(dataset) * (1 - test_size))
    print(split_idx)

    train_indices, test_indices = indices[:split_idx], indices[split_idx:]

    print('split done')

    train_dataset = Subset(dataset, train_indices)
    print(train_dataset)
    test_dataset = Subset(dataset, test_indices)
    print(test_dataset)

    return train_dataset, test_dataset

In [ ]:
print(type(adjacency_matrix))

In [ ]:
# import networkx as nx
# import numpy as np
# import torch
# from scipy.spatial.distance import pdist, squareform

# def spherical_to_cartesian(phi, theta):
#     phi_rad = np.radians(phi)
#     theta_rad = np.radians(theta)
#     x = np.cos(phi_rad) * np.sin(theta_rad)
#     y = np.sin(phi_rad) * np.sin(theta_rad)
#     z = np.cos(theta_rad)
#     return x, y, z

# def calculate_knn_graph(coordinates, k=5):
#     positions = [spherical_to_cartesian(coord['phi'], coord['theta']) for coord in coordinates.values()]
#     distance_matrix = squareform(pdist(positions))
#     knn_graph = nx.Graph()

#     for i, coord_i in enumerate(coordinates):
#         distances = distance_matrix[i]
#         nearest_indices = np.argsort(distances)[1:k+1]
#         for j in nearest_indices:
#             distance = distances[j]
#             weight = 1 / distance
#             knn_graph.add_edge(coord_i, list(coordinates.keys())[j], weight=weight)

#     for coord in coordinates:
#         knn_graph.add_edge(coord, coord, weight=1.0)

#     return knn_graph

# knn_graph = calculate_knn_graph(coordinates, k=5)

# print(knn_graph)

# edge_indices = []
# edge_weights = []

# for u, v, data in knn_graph.edges(data=True):
#     if u != v:
#         index_u = list(coordinates.keys()).index(u)
#         index_v = list(coordinates.keys()).index(v)
#         weight = data['weight']

#         edge_indices.extend([[index_u, index_v], [index_v, index_u]])
#         edge_weights.extend([weight, weight])

# for i in range(len(coordinates)):
#     edge_indices.append([i, i])
#     edge_weights.append(1.0)

# edge_index = torch.tensor(edge_indices, dtype=torch.long).t().contiguous()
# edge_weight = torch.tensor(edge_weights, dtype=torch.float)

# print(edge_index.shape)
# print(edge_weight.shape)

# eeg_dataset = EEGGraphDataset(data_df, edge_index, edge_weight)

# print(eeg_dataset[1])

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(device)

# for name, size in sorted(((name, sys.getsizeof(value)) for name, value in globals().items()),
#                         key= lambda x: -x[1])[:10]:
#     print("{:>30}: {:>8}".format(name, size))

# train_dataset, test_dataset = custom_train_test_split(eeg_dataset, test_size=0.2, random_state=42)

# train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, collate_fn=collate_graph_sequences)
# test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=collate_graph_sequences)


In [ ]:
G = nx.complete_graph(14)
edge_index = from_networkx(G).edge_index

weights = calculate_weights(coordinates)
edge_weights = torch.tensor(weights, dtype=torch.float)

eeg_dataset = EEGGraphDataset(data_df, edge_index, edge_weights)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

for name, size in sorted(((name, sys.getsizeof(value)) for name, value in globals().items()),
                        key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name, size))

train_dataset, test_dataset = custom_train_test_split(eeg_dataset, test_size=0.2, random_state=42)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, collate_fn=collate_graph_sequences)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=collate_graph_sequences)

del data_df

In [ ]:
class GRU(torch.nn.Module):
    def __init__(self, node_features):
        super(GRU, self).__init__()
        self.recurrent1 = GConvGRU(node_features, 64, 2)
        self.recurrent2 = GConvGRU(64, 32, 2)
        self.recurrent3 = GConvGRU(32, 16, 2)
        self.recurrent4 = GConvGRU(16, 8, 2)
        self.linear = torch.nn.Linear(8, 11)

    def forward(self, x, edge_index, edge_weight, batch):
        h = self.recurrent1(x, edge_index, edge_weight)
        h = F.relu(h)
        h = self.recurrent2(h, edge_index, edge_weight)
        h = F.relu(h)
        h = self.recurrent3(h, edge_index, edge_weight)
        h = F.relu(h)
        h = self.recurrent4(h, edge_index, edge_weight)
        h = F.relu(h)
        h = self.linear(h)
        graph_output = global_mean_pool(h, batch)
        return graph_output

class DC(torch.nn.Module):
    def __init__(self, node_features):
        super(DC, self).__init__()
        self.recurrent1 = DCRNN(node_features, 32, 1)
        self.recurrent2 = DCRNN(32, 16, 1)
        self.recurrent3 = DCRNN(16, 16, 1)
        self.linear = torch.nn.Linear(16, 11)

    def forward(self, x, edge_index, edge_weight, batch):
        h = self.recurrent1(x, edge_index, edge_weight)
        h = F.relu(h)
        h = self.recurrent2(h, edge_index, edge_weight)
        h = F.relu(h)
        h = self.recurrent3(h, edge_index, edge_weight)
        h = F.relu(h)
        h = self.linear(h)
        graph_output = global_mean_pool(h, batch)
        return graph_output


In [ ]:
print(device)

num_epochs = 100
node_features = 1

model = GRU(node_features).to(device)  # or DC(node_features)

optimizer = Adam(model.parameters(), lr=0.01)
criterion = CrossEntropyLoss()

def calculate_accuracy(y_pred, y_true):
    predicted = torch.argmax(y_pred, 1)
    correct = (predicted == y_true).float().sum()
    return (correct / y_true.shape[0]).item()

for epoch in tqdm(range(num_epochs)):
    model.train()
    total_loss = 0
    total_correct = 0
    all_labels = []
    all_preds = []

    for batched_sequences, labels in tqdm(train_loader):
        optimizer.zero_grad()
        sequence_outputs = []

        for sequence in batched_sequences:
          sequence.x = sequence.x.to(device)
          sequence.edge_index = sequence.edge_index.to(device)
          sequence.edge_weight = sequence.edge_attr.view(-1).to(device)
          sequence.batch = sequence.batch.to(device)
          graph_output = model(sequence.x, sequence.edge_index, sequence.edge_weight, sequence.batch)
          sequence_outputs.append(graph_output)

        sequence_outputs = torch.stack(sequence_outputs).mean(dim=0)
        labels = labels.to(device)
        labels = labels.long()
        loss = criterion(sequence_outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        # Calculate accuracy
        total_correct += calculate_accuracy(sequence_outputs, labels)
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(torch.argmax(sequence_outputs, 1).cpu().numpy())

    train_accuracy = total_correct / len(train_loader)
    train_confusion_matrix = confusion_matrix(all_labels, all_preds)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_loader)}, Train Accuracy: {train_accuracy}")
    print("Train Confusion Matrix:\n", train_confusion_matrix)

    # Validation
    model.eval()
    test_loss = 0
    total_correct = 0
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for batched_sequences, labels in test_loader:
            sequence_outputs = []

            for sequence in batched_sequences:
              sequence.x = sequence.x.to(device)
              sequence.edge_index = sequence.edge_index.to(device)
              sequence.edge_weight = sequence.edge_attr.view(-1).to(device)
              sequence.batch = sequence.batch.to(device)
              graph_output = model(sequence.x, sequence.edge_index, sequence.edge_weight, sequence.batch)
              sequence_outputs.append(graph_output)

            sequence_outputs = torch.stack(sequence_outputs).mean(dim=0)
            labels = labels.to(device)
            labels = labels.long()
            loss = criterion(sequence_outputs, labels)
            test_loss += loss.item()

            # Calculate accuracy
            total_correct += calculate_accuracy(sequence_outputs, labels)
            # all_labels.extend(labels.cpu().numpy())
            # all_preds.extend(torch.argmax(sequence_outputs, 1).cpu().numpy())

    test_accuracy = total_correct / len(test_loader)
    # test_confusion_matrix = confusion_matrix(all_labels, all_preds)
    print(f"Test Loss: {test_loss/len(test_loader)}, Test Accuracy: {test_accuracy}")
    # print("Test Confusion Matrix:\n", test_confusion_matrix)
